In [1]:
from IPython.display import display, set_matplotlib_formats
from collections import Counter
from itertools import chain
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from keras.callbacks import ModelCheckpoint
import pickle
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
import gensim,keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re,nltk
import seaborn as sns
import warnings
!pip install texttable
from texttable import Texttable  
set_matplotlib_formats('svg')
warnings.filterwarnings('ignore')
nltk.download('stopwords')
size_embedding = 200
windows = 2
min_count = 1
maxlen = 1000

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def clean_text(text):
    stop_words = set(stopwords.words('english'))
    to_remove = ['•', '!', '"', '#', '”', '“', '$', '%', '&', "'", '–', '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '…']
    stop_words.update(to_remove)
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub('\[[^]]*\]', '', text)
    text = (" ").join([word for word in text.split() if not word in stop_words])
    text = "".join([char for char in text if not char in to_remove])
    return text


In [3]:
def set_model(embedding_vectors):
    model = Sequential()
    model.add(Embedding(embedding_vectors.shape[0], output_dim=embedding_vectors.shape[1],weights=[embedding_vectors], input_length=maxlen, trainable=False))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
def w2v_to_keras_weights(model, vocab):
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, size_embedding))
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [5]:
def createTabel(Test_loss,Test_accu,Train_loss,Train_accu):
  rows = [["Parameters", "Accuracy", "Loss"], ["Training", Train_accu, Train_loss], ["Testing", Test_accu, Test_loss]]
  table = Texttable()
  table.add_rows(rows)
  print(table.draw())

In [6]:
def plot_loss_epochs(history):
    epochs = np.arange(1,len(history.history['accuracy']) + 1,1)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']

    fig , ax = plt.subplots(1,2, figsize=(7,3))
    ax[0].plot(epochs , train_acc , '.-' , label = 'Train Accuracy')
    ax[0].plot(epochs , val_acc , '.-' , label = 'Validation Accuracy')
    ax[0].set_title('Train & Validation Accuracy')
    ax[0].legend()
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Accuracy")

    ax[1].plot(epochs , train_loss , '.-' , label = 'Train Loss')
    ax[1].plot(epochs , val_loss , '.-' , label = 'Validation Loss')
    ax[1].set_title('Train & Validation Loss')
    ax[1].legend()
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Loss")
    fig.tight_layout()
    fig.show()

In [7]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/train.csv')
df['Text']=df['title']+" "+df['text']
df = df.drop(columns = ['title', 'text','author'])
df['Text'].fillna("NA", inplace=True)
df['Text'] = df['Text'].apply(clean_text)

In [8]:
text_train=df['Text']
y_train=df['label']
text_train_splited = [article.split() for article in text_train]
w2v_model = gensim.models.Word2Vec(sentences = text_train_splited, size = size_embedding, window = windows, min_count = min_count)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train_splited)
text_train_tok = tokenizer.texts_to_sequences(text_train_splited)
word_index = tokenizer.word_index

embedding_vectors = w2v_to_keras_weights(w2v_model, word_index) ##Embedding Vectors

text_train_tok_pad = pad_sequences(text_train_tok, maxlen=maxlen)


In [9]:
model = set_model(embedding_vectors = embedding_vectors)
model.summary()
checkpoint = ModelCheckpoint("best_weights.hdf5", monitor='accuracy', verbose=0,save_best_only=True, mode='auto', period=1,save_weights_only=False) #checkpoint at best epoch
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(text_train_tok_pad, y_train, validation_split=0.05,callbacks=[checkpoint], epochs=15, batch_size = 64, verbose = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 200)         44166400  
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                59648     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 44,226,113
Trainable params: 59,713
Non-trainable params: 44,166,400
_________________________________________________________________
Epoch 1/15
309/309 [==============================] - 35s 83ms/step - loss: 0.4479 - accuracy: 0.7964 - val_loss: 0.3120 - val_accuracy: 0.8654
Epoch 2/15
309/309 [==============================] - 24s 79ms/step - loss: 0.2381 - accuracy: 0.9079 - val_loss: 0.2156 - val_accuracy: 0.9192
Epoch 3/15
309/309 [==============================] - 24s 79ms/step - loss: 0.177

In [10]:
dfTest=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ML/test.csv')
dfTest['Text']=dfTest['title']+" "+dfTest['text']
dfTest = dfTest.drop(columns = ['title', 'text','author'])
dfTest['Text'].fillna("NA", inplace=True)
dfTest['Text'] = dfTest['Text'].apply(clean_text)
text_test=dfTest['Text']

text_test_splited = [article.split() for article in text_test]
text_test_tok = tokenizer.texts_to_sequences(text_test_splited)
text_test_tok_pad = pad_sequences(text_test_tok, maxlen=maxlen)

In [12]:
predicted = (model.predict(text_test_tok_pad) > 0.5).astype("int32")
id=dfTest['id']
ll=predicted.tolist()
import csv
row_list=[['id','label']]
for i in range(len(id)):
  row_list+=[[id[i],ll[i][0]]]

filename='submission_w2v_BiLSTM.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)